# merging of abdominal Scans from UKB

In [1]:
import os

import nibabel  # for loading and saving nifty files
import torch
from math import ceil, floor
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import Pad
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

In [3]:
print(os.getcwd())

/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/datasets


### Combine Images function
opp_files = list of all opp files that was found for specific patient  
folder = directory of the files  
data_dir = data directory of the files, (one level above folder)  
sequence = 'opp' or 'in' etc... in our case 'opp'  

loads all opp scans for the patient. Then looks at the header information - specifically qoffset_z, and takes the upper 2 scans (according to qoffset_z value)  
then calculates the overlap between the 2 scans and size of the new combined volume  

within the overlap region we use a linear function to merge the 2 scans:

for a pixel pix with pixel location p, the new value is calculated as:    
new_pix = (1-a*p) * im_0_pix + (a*p) * im_1_pix  
where a = 1/overlap and overlap is the siye 






In [9]:
def combine_images(opp_files, data_dir,sequence):
    
    images = []
    # load files
    for file in opp_files:
        if "nii.gz" in file:
            img = nibabel.load(os.path.join(data_dir, file))
            #print(img)
            img_h = img.header
            print('x: ', img_h['qoffset_x'])
            print('y: ', img_h['qoffset_y'])
            print('z: ', img_h['qoffset_z'])
            #print(img_h.get_data_shape())
            images.append(img)
    #print(images)
    
    print('len images ', len(images))
    if len(images) < 3:
        print('less than 3 images')
        return

    # sort according to qoffset_z
    images_sorted = sorted(images, key=lambda im: im.header['qoffset_z'], reverse=True)

    #print(images_sorted[0].header['qoffset_z'])

    #take the upper 2 scans
    im_0 = images_sorted[0]
    im_1 = images_sorted[1]

    im_0_dim_v = im_0.shape[2]
    im_1_dim_v = im_1.shape[2]

    #print(im_0_dim_v)

    # calculate overlap region:
    im_0_end = im_0.header['qoffset_z']
    im_1_end = im_1.header['qoffset_z']
    print('im_0_end, im_1,end: ', im_0_end, im_1_end)

    spacing = im_0.header['pixdim'][3]
    print('pixel spacing: ', spacing)

    im_0_dim_w = im_0_dim_v * spacing
    im_1_dim_w = im_1_dim_v * spacing

    im_1_start = im_1_end + im_1_dim_w
    im_0_start = im_0_end + im_0_dim_w
    print('im_0_start, im_1,start: ', im_1_start, im_0_start)

    overlap = abs(im_0_end - im_1_start)
    print('overlap: ', overlap)

    overlap_v = int(round(overlap/spacing))
    print('overlap_v: ', overlap_v)

    new_im_dim = round((abs(im_1_end - im_0_end) + abs(im_0_end - im_0_start))/spacing)
    print('new im dim: ', new_im_dim)
    
    new_img = np.empty([im_0.shape[0], im_0.shape[1], int(new_im_dim)])
    print('new img shape: ', new_img.shape)
    im_0_data = im_0.get_data()
    im_1_data = im_1.get_data()


    # bottom ( origin is bottom left)    
    new_img[:,:,0:(im_1_dim_v-overlap_v)] = im_1_data[:,:,0:(im_1_dim_v-overlap_v)]    
    # top:
    new_img[:,:, im_1_dim_v:] = im_0_data[:,:,overlap_v:]
    
    # overlap region:
    a = 1/overlap_v
    print('overlap_v ', overlap_v)
    print('a ', a)
    for l in range(0,overlap_v):
        new_img[:,:,(im_1_dim_v-overlap_v+l)] = (1-a*l) * im_1_data[:,:,(im_1_dim_v-overlap_v)+l] + (a*l) * im_0_data[:,:,l]

    empty_header = nibabel.Nifti1Header()
    new_img_nii = nibabel.Nifti1Image(new_img, im_1.affine, empty_header)
    new_img_nii.header['pixdim'] = im_1.header['pixdim']

    print('new header: ', new_img_nii.header)
    new_file_path = os.path.join(data_dir,'Dixon_BH_17s_%s_Dixon_BH_17_comb_01.nii.gz'%sequence)

    nibabel.save(new_img_nii, new_file_path)

Define data directory in data_dir. Gets all Dixon BH 17s opp files and calls combine_images function.

In [10]:
data_dir = '/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/datasets/UKB/3387088_20201_2_0'
#listFolders =  os.listdir(data_dir)
#for folder in listFolders:
#path = data_dir + folder
listFiles = os.listdir(data_dir)
sequence = 'opp'
opp_files = [f for f in listFiles if "Dixon_BH_17s_%s_Dixon_BH_17"%sequence in f]
#print(opp_files)

combine_images(opp_files, data_dir,sequence)
            
            
    

x:  -247.76788
y:  -191.9643
z:  -462.75
x:  -247.76788
y:  -191.9643
z:  -294.75
x:  -247.76788
y:  -191.9643
z:  -462.75
x:  -247.76788
y:  -191.9643
z:  -630.75
len images  4
im_0_end, im_1,end:  -294.75 -462.75
pixel spacing:  4.5
im_0_start, im_1,start:  -264.75 -96.75
overlap:  30.0
overlap_v:  7
new im dim:  81.0
new img shape:  (224, 174, 81)
overlap_v  7
a  0.14285714285714285
new header:  <class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 224 174  81   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.       2.232143 2.232143 4.5      0.00669  0.       0.       0.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_c